In [1]:
import time
import numpy as np
import pandas as pd
from multiprocessing import Pool
from analyze_weather import analyze_temperature

In [2]:
def benchmark(func, *args, **kwargs):
    start_time = time.time()
    result = func(*args, **kwargs)
    
    end_time = time.time()
    print(f"{func.__name__} выполнено за {end_time - start_time:.2f} секунд")
    return result

In [3]:
data = pd.read_csv('temperature_data.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])

In [4]:
def analyze_temperature_sequential(data):
    window = 30
    cities = data['city'].unique()
    stats = []
    for city in cities:
        stats.append(analyze_temperature(data, city, window))
    return stats

result = benchmark(analyze_temperature_sequential, data)

analyze_temperature_sequential выполнено за 0.07 секунд


In [5]:
def analyze_temperature_multiproc(data):
    window = 30
    cities = data['city'].unique()
    args = [(data, city, window) for city in cities]
    with Pool() as pool:
        stats = pool.starmap(analyze_temperature, args)
    return stats

result = benchmark(analyze_temperature_multiproc, data)

analyze_temperature_multiproc выполнено за 2.14 секунд


multiprocessing.Pool отработал медленнее чем последовательное выполнение. Скорее всего это связано с тем, что вычислительная задача слишком простая, и накладные расходы на управление процессами превышают выигрыш от параллельного выполнения.

In [6]:
import asyncio
import aiohttp
import nest_asyncio
import requests
nest_asyncio.apply()

from analyze_weather import get_all_temperatures

In [7]:
def get_all_temperatures_async(cities, api_key):
    current_temperatures = asyncio.run(get_all_temperatures(cities, api_key))
    return current_temperatures

cities = data['city'].unique()
api_key = '...'
result = benchmark(get_all_temperatures_async, cities, api_key)

get_all_temperatures_async выполнено за 0.21 секунд


In [8]:
def get_temperature_sync(city, api_key):
    url = f"https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}&units=metric"
    response = requests.get(url)
    data = response.json()
    if data['cod'] == 200:
        return city, data['main']['temp']
    else:
        raise ValueError(str(data))


def get_all_temperatures_sync(cities, api_key):
    results = []
    for city in cities:
        results.append(get_temperature_sync(city, api_key))
    return dict(results)

result = benchmark(get_all_temperatures_sync, cities, api_key)

get_all_temperatures_sync выполнено за 2.12 секунд


Асинхронное выполнение запросов к API отработало быстрее, чем синхронное.